In [20]:
# Create the directory if it doesn't exist
!mkdir C:\Users\jplv0\PycharmProjects\nyc-time-prediction\data

# Download files using curl
!curl -o C:\Users\jplv0\PycharmProjects\nyc-time-prediction/data/green_tripdata_2024-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet
!curl -o C:\Users\jplv0\PycharmProjects\nyc-time-prediction/data/green_tripdata_2024-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1330k  100 1330k    0     0  5574k      0 --:--:-- --:--:-- --:--:-- 5589k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1253k  100 1253k    0     0  6357k      0 --:--:-- --:--:-- --:--:-- 6396k


In [21]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
from sklearn.linear_model import Lasso, Ridge, LinearRegression

In [22]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [24]:
df_train = read_dataframe('C:/Users/jplv0/PycharmProjects/nyc-time-prediction/data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('C:/Users/jplv0\PycharmProjects/nyc-time-prediction/data/green_tripdata_2024-02.parquet')

In [25]:
df_train

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
0,2,2024-01-01 00:46:55,2024-01-01 00:58:25,N,1.0,236,239,1.0,1.98,12.80,...,0.5,3.61,0.0,NaN,1.0,21.66,1.0,1.0,2.75,11.500000
1,2,2024-01-01 00:31:42,2024-01-01 00:52:34,N,1.0,65,170,5.0,6.54,30.30,...,0.5,7.11,0.0,NaN,1.0,42.66,1.0,1.0,2.75,20.866667
2,2,2024-01-01 00:30:21,2024-01-01 00:49:23,N,1.0,74,262,1.0,3.08,19.80,...,0.5,3.00,0.0,NaN,1.0,28.05,1.0,1.0,2.75,19.033333
3,1,2024-01-01 00:30:20,2024-01-01 00:42:12,N,1.0,74,116,1.0,2.40,14.20,...,1.5,0.00,0.0,NaN,1.0,16.70,2.0,1.0,0.00,11.866667
4,2,2024-01-01 00:32:38,2024-01-01 00:43:37,N,1.0,74,243,1.0,5.14,22.60,...,0.5,6.28,0.0,NaN,1.0,31.38,1.0,1.0,0.00,10.983333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56546,2,2024-01-31 20:46:00,2024-01-31 20:55:00,None,NaN,33,25,NaN,0.00,11.58,...,0.0,3.14,0.0,NaN,1.0,15.72,NaN,NaN,NaN,9.000000
56547,2,2024-01-31 21:06:00,2024-01-31 21:11:00,None,NaN,72,72,NaN,0.49,11.58,...,0.0,0.00,0.0,NaN,1.0,12.58,NaN,NaN,NaN,5.000000
56548,2,2024-01-31 21:36:00,2024-01-31 21:40:00,None,NaN,72,72,NaN,0.52,11.58,...,0.0,2.52,0.0,NaN,1.0,15.10,NaN,NaN,NaN,4.000000
56549,2,2024-01-31 22:45:00,2024-01-31 22:51:00,None,NaN,41,42,NaN,1.17,14.22,...,0.0,0.00,0.0,NaN,1.0,15.22,NaN,NaN,NaN,6.000000


In [26]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [27]:
df_train.columns


Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge',
       'duration', 'PU_DO'],
      dtype='object')

In [28]:
categorical = ['PU_DO']  #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [29]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [30]:
import dagshub
import mlflow


dagshub.init(url="https://dagshub.com/juanplv04/nyc-taxi-time-prediction", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

Accessing as juanplv04

Repository nyc-taxi-time-prediction doesn't exist, creating it under current user.

Initialized MLflow to track repo "juanplv04/nyc-taxi-time-prediction"

Repository juanplv04/nyc-taxi-time-prediction initialized!

https://dagshub.com/juanplv04/nyc-taxi-time-prediction.mlflow


2024/09/27 13:53:46 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/ea138581011246d2b8b3f5a27083e539', creation_time=1727466826893, experiment_id='0', last_update_time=1727466826893, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [34]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2024-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2024-02")

In [33]:
from dagshub import get_repo_bucket_client
# Get a boto3.client object
s3 = get_repo_bucket_client("juanplv04/nyc-taxi-time-prediction")

# Upload file
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-01.parquet",  # local path of file to upload
    Key="train_data.parquet",  # remote path where to upload the file
)

## test
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-02.parquet",  # local path of file to upload
    Key="eval_data.parquet",  # remote path where to upload the file
)


Client created. Use the name of the repo (nyc-taxi-time-prediction) as the name of the bucket

In [35]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import pathlib

In [36]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [37]:
def objective(params):
    with mlflow.start_run(nested=True):
         
        # Tag model
        mlflow.set_tag("model_family", "xgboost")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Train model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10
        )
        
        # Log xgboost model with artifact_path
        mlflow.xgboost.log_model(booster, artifact_path="model")
         
        # Predict in the val dataset
        y_pred = booster.predict(valid)
        
        # Calculate metric
        rmse = root_mean_squared_error(y_val, y_pred)
        
        # Log performance metric
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [38]:
mlflow.xgboost.autolog()

with mlflow.start_run(run_name="Xgboost Hyper-parameter Optimization", nested=True):
    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
        'objective': 'reg:squarederror',
        'seed': 42
    }
    
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"
    
    mlflow.log_params(best_params)

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "NYC Taxi Time Prediction Project",
            "optimizer_engine": "hyper-opt",
            "model_family": "xgboost",
            "feature_set_version": 1,
        }
    )

    # Log a fit model instance
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )
        
    y_pred = booster.predict(valid)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    pathlib.Path("models").mkdir(exist_ok=True)
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

[0]	validation-rmse:6.99522                                                                                                                                                             
[1]	validation-rmse:6.01012                                                                                                                                                             
[2]	validation-rmse:5.58650                                                                                                                                                             
[3]	validation-rmse:5.40396                                                                                                                                                             
[4]	validation-rmse:5.31090                                                                                                                                                             
[5]	validation-rmse:5.27226                                                

[44]	validation-rmse:5.17415                                                                                                                                                            
[45]	validation-rmse:5.17310                                                                                                                                                            
[46]	validation-rmse:5.17301                                                                                                                                                            
[47]	validation-rmse:5.17309                                                                                                                                                            
[48]	validation-rmse:5.17330                                                                                                                                                            
[49]	validation-rmse:5.17338                                               

2024/09/27 13:58:45 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/27 13:58:45 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\jplv0\anaconda3\lib\site-packages\xgboost\core.py:160: UserWarning: [13:58:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."

2024/09/27 13:58:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\jplv0\anaconda3\lib\site-packages\xgboost\core.py:160: UserWarning: [13:58:52] WARNING: C:\buildkite-

[0]	validation-rmse:7.59016                                                                                                                                                             
[1]	validation-rmse:6.64720                                                                                                                                                             
[2]	validation-rmse:6.08726                                                                                                                                                             
[3]	validation-rmse:5.76744                                                                                                                                                             
[4]	validation-rmse:5.58300                                                                                                                                                             
[5]	validation-rmse:5.48089                                                

[44]	validation-rmse:5.27214                                                                                                                                                            
[45]	validation-rmse:5.27075                                                                                                                                                            
[46]	validation-rmse:5.26987                                                                                                                                                            
[47]	validation-rmse:5.27011                                                                                                                                                            
[48]	validation-rmse:5.26985                                                                                                                                                            
[49]	validation-rmse:5.26849                                               

...

[0]	validation-rmse:5.93542                                                                                                                                                             
[1]	validation-rmse:5.38253                                                                                                                                                             
[2]	validation-rmse:5.28178                                                                                                                                                             
[3]	validation-rmse:5.24523                                                                                                                                                             
[4]	validation-rmse:5.23755                                                                                                                                                             
[5]	validation-rmse:5.23326                                                

[44]	validation-rmse:5.17796                                                                                                                                                            
[45]	validation-rmse:5.17843                                                                                                                                                            
[46]	validation-rmse:5.17674                                                                                                                                                            
[47]	validation-rmse:5.17833                                                                                                                                                            
[48]	validation-rmse:5.17944                                                                                                                                                            
[49]	validation-rmse:5.17837                                               

2024/09/27 14:04:57 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/27 14:04:57 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\jplv0\anaconda3\lib\site-packages\xgboost\core.py:160: UserWarning: [14:04:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."

2024/09/27 14:04:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\jplv0\anaconda3\lib\site-packages\xgboost\core.py:160: UserWarning: [14:05:02] WARNING: C:\buildkite-

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [06:34<00:00, 39.48s/trial, best loss: 5.156129557204716]
[0]	validation-rmse:6.99522
[1]	validation-rmse:6.01012
[2]	validation-rmse:5.58650
[3]	validation-rmse:5.40396
[4]	validation-rmse:5.31090
[5]	validation-rmse:5.27226
[6]	validation-rmse:5.25420
[7]	validation-rmse:5.23930
[8]	validation-rmse:5.23068
[9]	validation-rmse:5.22769
[10]	validation-rmse:5.22529
[11]	validation-rmse:5.22353
[12]	validation-rmse:5.21934
[13]	validation-rmse:5.21576
[14]	validation-rmse:5.21381
[15]	validation-rmse:5.21208
[16]	validation-rmse:5.20880
[17]	validation-rmse:5.20266
[18]	validation-rmse:5.20058
[19]	validation-rmse:5.20026
[20]	validation-rmse:5.19786
[21]	validation-rmse:5.19801
[22]	validation-rmse:5.19725
[23]	validation-rmse:5.19607
[24]	validation-rmse:5.19343
[25]	validation-rmse:5.19181
[26]	validation-rmse:5.18950
[27]	validation-rmse:5.18854
[28]	validati

2024/09/27 14:05:18 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/09/27 14:05:18 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\jplv0\anaconda3\lib\site-packages\xgboost\core.py:160: UserWarning: [14:05:18] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
2024/09/27 14:05:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/27 14:05:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run Xgboost Hyper-parameter Optimizatio

In [39]:
best_params

{'learning_rate': 0.39273744441544745,
 'max_depth': 28,
 'min_child_weight': 1.0378854742772095,
 'reg_alpha': 0.15509121921771926,
 'reg_lambda': 0.3337934970235596,
 'seed': 42,
 'objective': 'reg:squarederror'}

In [20]:
import mlflow

In [42]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

Ingrese el run_id5eccd30b127c464596379d95bf2b434e


Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
2024/09/27 14:13:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-model, version 2
Created version '2' of model 'nyc-taxi-model'.


In [41]:
from datetime import datetime
from mlflow import MlflowClient

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="nyc-taxi-model",
    description="Model registry for the NYC Taxi Time Prediction Project",
)

new_alias = "champion"
date = datetime.today()
model_version = "1"

# create "champion" alias for version 1 of model "nyc-taxi-model"
client.set_registered_model_alias(
    name="nyc-taxi-model",
    alias=new_alias,
    version=model_version
)

client.update_model_version(
    name="nyc-taxi-model",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=['campion', 'champion'], creation_timestamp=1727467720762, current_stage='None', description='The model version 1 was transitioned to champion on 2024-09-27 14:09:54.788432', last_updated_timestamp=1727467795284, name='nyc-taxi-model', run_id='5eccd30b127c464596379d95bf2b434e', run_link='', source='mlflow-artifacts:/ea138581011246d2b8b3f5a27083e539/5eccd30b127c464596379d95bf2b434e/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [23]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

2024/09/20 17:25:06 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.16.2, required: mlflow==2.16.1)
 - scikit-learn (current: 1.5.1, required: scikit-learn==1.5.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


array([17.868977 , 27.244492 ,  7.5599394, ..., 36.802193 , 10.982415 ,
       19.203161 ], dtype=float32)